In [ ]:
# investing site에서 정보를 크롤링하는 코드 + db에 적재하는 코드이다.

# 위 코랩상에선는 크롤링이 잘 수행이 되고 잘 돌아가나 , vs code로 하였을떄는 안되는 것을 파악했다.

# 이로써 우리 회사의 ip를 차단했다는 것을 알 수 있는 좋은 경험이었다.

import requests
from bs4 import BeautifulSoup
import logging

import time
from datetime import datetime, date, timedelta

from fastapi import APIRouter, Depends
from fastapi_utils.cbv import cbv
from sqlalchemy import func, insert, select ,update
from sqlalchemy.ext.asyncio import AsyncSession


from common.db import db_etc
from models.models_etc import InvestingPmiMonthly
import pymysql


In [ ]:

router = APIRouter()
logger = logging.getLogger('uvicorn')


In [ ]:

@cbv(router=router)
class investingpmi:
    db:AsyncSession = Depends(db_etc.get_db)


   # 크롤링 수행 후 테스트 
    @router.get('/chinapmi',
        summary='china pmi information')

    
    async def caixin_china(self):

        try:

            URL = 'https://kr.investing.com/economic-calendar/chinese-caixin-manufacturing-pmi-753'
 
            res = requests.get(URL ,headers={"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 "})
            print(res)
            soup = BeautifulSoup(res.text,'html.parser')

            table = soup.find('table' , 'genTbl openTbl ecHistoryTbl')
            
        
            if table != None:
                tbody =table.find('tbody')
                
                
                trs = tbody.find_all('tr')

                results =[]
                for tr in trs:
                    
                    tds = tr.find_all('td')
                    
                    item = {
                        'release date':f'{tds[0].text}',
                        'time' : f'{tds[1].text}',
                        'real' : f'{tds[2].text}',
                        'prediction' : f'{tds[3].text}',
                        'before' : f'{tds[4].text}',
                        
                        }

            
                    results.append(item)
            
                return results

        except Exception:
            logger.exception('중국 pmi')


    async def get_origin_investing(self):
      try:
        q = select(InvestingPmiMonthly)
        rs = await self.db.execute(q)
        return rs.scalars().all()

      except Exception as e:
            print(e)

    


    # 데이터 베이스 적재 코드


    @router.post('/updatechinapmi',
        summary='china pmi update 후 적재')


    async def update_caixin_china(self):

        try:

            crawl = await self.caixin_china()
            original = await self.get_origin_investing()

            for c in crawl:
                if c["release_date"] == datetime.today: 
                    # 가장 오래된 datetime , 언제 업데이트를 실행하게 될지 확실하지 않기 때문에 에어플로우를 
                    # 에어플로우 기간을 업데이트 될것 같은 시기 쯤 일주일 정도를 해놓고 맞는 값이 있는지 확인해주자 
                    original["release_date"] = c["release_date"]
                    original["time"] = datetime.now
                    original["real"] = c["real"]
                    original["prediction"] = c["prediction"]
                    original["before"] = c["before"]



                await self.db.commit()

        except Exception as e:
            print(f"error : {e.with_traceback()}")
            return None



                

            





        